In [38]:
import numpy as np
import numpy.linalg as la
from typing import Callable
import matplotlib.pyplot as plt
import sympy as sp
import pandas as pd
pd.set_option("display.precision", 8)


# Newton Non-Linear equation 

$$f(\mathbf x) = \begin{bmatrix}3x_1 - \cos{x_2 x_3} - 0.5\\x_1^2 - 81{(x_2 + 0.1)}^2 + \sin{x_3} + 1.06\\e^{-x_1x_2} + 20x_3 + \frac{10\pi - 3}{3} \end{bmatrix}$$

In [39]:
# x0 = np.array([0.1, 0.1, 0.1])
f0 = lambda x: np.array([3 * x[0] - np.cos(x[1] * x[2]) - 0.5,
                         x[0]**2 - 81.0 * (x[1] + 0.1)**2 + np.sin(x[2]) + 1.06,
                         np.e**(-x[0] * x[1]) + 20 * x[2] + (10 * np.pi - 3.0) / 3.0])

df0 = lambda x: np.array([[3, x[2] * np.sin(x[1] * x[2]), x[1] * np.sin(x[1] * x[2])],
                          [2 * x[0], -162 * (x[1] + 0.1), np.cos(x[2])],
                          [-x[1] * np.e**(-x[0] * x[1]), -x[0] * np.e**(-x[0] * x[1]), 20]])


In [40]:
def newton_method(f: Callable, df: Callable, x0: np.ndarray, MAX_ITER=100, tol=1e-8):
    """
    Newton's method to solve the system f(x)=0
    """
    for k in range(1, MAX_ITER):
        f_val = f(x0)
        df_val = df(x0)
        # update step
        dx = np.linalg.solve(df_val, f_val)
        x = x0 - dx
        # absolute error 
        abs_err = la.norm(x - x0, np.inf)
        # print step
        print(f"{k:02d} {x} {abs_err:0.8f}")
        # stop criterion |x-x0|=0
        if abs_err < tol:
            return x
        x0 = x.copy()
    print("Too many iterations")
    return None

In [41]:
x0 = np.array([1.0, 1.0, 1.0])
np.set_printoptions(precision=8, suppress=True)
print("k  x[k]\t\t\t\t\t error absolute")
newton_method(f0, df0, x0);


k  x[k]					 error absolute
01 [ 0.91968721  0.46082246 -0.50338764] 1.50338764
02 [ 0.50100049  0.18743348 -0.52086923] 0.41868673
03 [ 0.50054294  0.06115345 -0.52200096] 0.12628002
04 [ 0.50010444  0.01161711 -0.52329515] 0.04953635
05 [ 0.50000551  0.00060562 -0.52358294] 0.01101149
06 [ 0.50000002  0.00000183 -0.52359873] 0.00060379
07 [ 0.5         0.         -0.52359878] 0.00000183
08 [ 0.5        -0.         -0.52359878] 0.00000000


# Newton Optimization

Iteración
$$x_{k+1}=x_k - {[\nabla ^2 f(x_k)]}^{-1} \nabla  f(x_k)$$

In [42]:
def newton_optimization(f: Callable, df: Callable, ddf: Callable, x: np.ndarray, MAX_ITER=100, tol=1e-7):
    """
    Newton's method to solve min f(x) for x in R^n
    """
    print(f"{0:02d} {x}")
    for k in range(1, MAX_ITER):
        # calculate gradient and hessian
        gradient = df(x)
        hessian = ddf(x)
        # update step
        dx = np.linalg.solve(hessian, -gradient)
        x = x + dx
        # gradient norm error
        grad_err = la.norm(df(x), np.inf)
        # print step
        print(f"{k:02d} {x} {f(x):0.9f} {grad_err:0.9f}")
        # stop criterion |∇f|=0
        if grad_err < tol:
            return x
    print("Too many iterations")
    return None


# Laboratorio Semana 11: Método de Newton

Aplicando el método de Newton Minimizar las siguientes funciones

##### a) $f(\mathbf x) = 7x_1^2 + 2x_1x_2 + x_2^2$

In [43]:
fa = lambda x: 7 * x[0]**2 + 2 * x[0] * x[1] + x[1]**2
dfa = lambda x: np.array([14 * x[0] + 2 * x[1], 2 * x[0] + 2 * x[1]])
ddfa = lambda x: np.array([[14, 2],
                           [2, 2.]])

x0 = np.array([-2, 2.])
print("k  x[k]\tf(x[k])\terror |∇f|=0")
newton_optimization(fa, dfa, ddfa, x0);


k  x[k]	f(x[k])	error |∇f|=0
00 [-2.  2.]
01 [0. 0.] 0.000000000 0.000000000


##### b) $f(\mathbf x)=x_{1}^{2} + x_{1} x_{2} + 2 x_{1} x_{3} + 4 x_{2}^{2} + x_{2} x_{3} + 4 x_{3}^{2}$

In [44]:
fb = lambda x: x[0]**2 + x[0] * x[1] + 2 * x[0] * x[2] + 4 * x[1]**2 + x[1] * x[2] + 4 * x[2]**2
dfb = lambda x: np.array([2 * x[0] + x[1] + 2 * x[2], x[0] + 8 * x[1] + x[2], 2 * x[0] + x[1] + 8 * x[2]])
ddfb = lambda x: np.array([[2, 1, 2],
                           [1, 8, 1],
                           [2, 1, 8.]])
                           
x0 = np.array([1.0, 1.0, 1.0])
print("k  x[k]\tf(x[k])\terror |∇f|=0")
newton_optimization(fb, dfb, ddfb, x0);


k  x[k]	f(x[k])	error |∇f|=0
00 [1. 1. 1.]
01 [0. 0. 0.] 0.000000000 0.000000000


##### c) $f(\mathbf x)=4 \left(x_{1} - 2\right)^{4} + 2 \left(x_{1} - 2 x_{2}\right)^{2}$

In [45]:
fc = lambda x: 4 * (x[0] - 2)**4 + 2 * (x[0] - 2 * x[1])**2
dfc = lambda x: np.array([4 * x[0] - 8 * x[1] + 16 * (x[0] - 2)**3, -8 * x[0] + 16 * x[1]])
ddfc = lambda x: np.array([[48 * (x[0] - 2)**2 + 4, -8],
                           [-8, 16]])
x0 = np.array([1.0, 1.2])
print("k\tx[k]\t\tf(x[k])\terror |∇f|=0")
newton_optimization(fc, dfc, ddfc, x0);


k	x[k]		f(x[k])	error |∇f|=0
00 [1.  1.2]
01 [1.33333333 0.66666667] 0.790123457 4.740740741
02 [1.55555556 0.77777778] 0.156073769 1.404663923
03 [1.7037037  0.85185185] 0.030829387 0.416196718
04 [1.80246914 0.90123457] 0.006089755 0.123317546
05 [1.86831276 0.93415638] 0.001202915 0.036538532
06 [1.9122085  0.95610425] 0.000237613 0.010826232
07 [1.94147234 0.97073617] 0.000046936 0.003207772
08 [1.96098156 0.98049078] 0.000009271 0.000950451
09 [1.97398771 0.98699385] 0.000001831 0.000281615
10 [1.98265847 0.99132924] 0.000000362 0.000083442
11 [1.98843898 0.99421949] 0.000000071 0.000024723
12 [1.99229265 0.99614633] 0.000000014 0.000007325
13 [1.99486177 0.99743088] 0.000000003 0.000002171
14 [1.99657451 0.99828726] 0.000000001 0.000000643
15 [1.99771634 0.99885817] 0.000000000 0.000000191
16 [1.99847756 0.99923878] 0.000000000 0.000000056


##### d) $f(\mathbf x)=\left(x_{1} - 2\right)^{4} + \left(x_{1} - x_{2}\right)^{2}$


In [46]:
fd = lambda x: (x[0] - 2)**4 + (x[0] - x[1])**2
dfd = lambda x: np.array([2 * x[0] - 2 * x[1] + 4 * (x[0] - 2)**3, -2 * x[0] + 2 * x[1]])
ddfd = lambda x: np.array([[12 * (x[0] - 2)**2 + 2, -2], [-2, 2]])

x0 = np.array([1.0, 1.2])
print("k\tx[k]\t\tf(x[k])\terror |∇f|=0")
newton_optimization(fd, dfd, ddfd, x0);


k	x[k]		f(x[k])	error |∇f|=0
00 [1.  1.2]
01 [1.33333333 1.33333333] 0.197530864 1.185185185
02 [1.55555556 1.55555556] 0.039018442 0.351165981
03 [1.7037037 1.7037037] 0.007707347 0.104049179
04 [1.80246914 1.80246914] 0.001522439 0.030829387
05 [1.86831276 1.86831276] 0.000300729 0.009134633
06 [1.9122085 1.9122085] 0.000059403 0.002706558
07 [1.94147234 1.94147234] 0.000011734 0.000801943
08 [1.96098156 1.96098156] 0.000002318 0.000237613
09 [1.97398771 1.97398771] 0.000000458 0.000070404
10 [1.98265847 1.98265847] 0.000000090 0.000020860
11 [1.98843898 1.98843898] 0.000000018 0.000006181
12 [1.99229265 1.99229265] 0.000000004 0.000001831
13 [1.99486177 1.99486177] 0.000000001 0.000000543
14 [1.99657451 1.99657451] 0.000000000 0.000000161
15 [1.99771634 1.99771634] 0.000000000 0.000000048


##### e) $f(\mathbf x)=x_{1}^{4} + \left(x_{1} + x_{2}\right)^{2} + \left(e^{x_{2}} - 1\right)^{2}$


In [47]:
fe = lambda x: x[0]**4 + (x[0] + x[1])**2 + (np.exp(x[1]) - 1)**2
dfe = lambda x: np.array([4 * x[0]**3 + 2 * x[0] + 2 * x[1], 2 * x[0] + 2 * x[1] + 2 * (np.exp(x[1]) - 1) * np.exp(x[1])])
ddfe = lambda x: np.array([[12 * x[0]**2 + 2, 2],
                           [2, 2 * (np.exp(x[1]) - 1) * np.exp(x[1]) + 2 * np.exp(2 * x[1]) + 2]])

x0 = np.array([1.0, 1.2])
print("k\tx[k]\t\tf(x[k])\terror |∇f|=0")
newton_optimization(fe, dfe, ddfe, x0);


k	x[k]		f(x[k])	error |∇f|=0
00 [1.  1.2]
01 [0.46786411 0.72495126] 2.604162320 6.781767668
02 [0.03509768 0.32846156] 0.283365491 1.807155485
03 [-0.10836193  0.10933579] 0.013487599 0.259718981
04 [-0.02164563  0.01808097] 0.000345822 0.030027320
05 [-0.00055387  0.00051486] 0.000000267 0.000952501
06 [-0.0000004  0.0000004] 0.000000000 0.000000795
07 [-0.  0.] 0.000000000 0.000000000


##### f) $f(\mathbf x)=\frac{3 x_{1}^{2}}{2} + x_{1} x_{3} - x_{1} + 2 x_{2}^{2} + 2 x_{2} x_{3} - x_{2} + \frac{3 x_{3}^{2}}{2} - x_{3} + 3$


In [48]:
ff = lambda x: 3 * x[0]**2 / 2 + x[0] * x[2] - x[0] + 2 * x[1]**2 + 2 * x[1] * x[2] - x[1] + 3 * x[2]**2 / 2 - x[2] + 3
dff = lambda x: np.array([3 * x[0] + x[2] - 1, 4 * x[1] + 2 * x[2] - 1, x[0] + 2 * x[1] + 3 * x[2] - 1])
ddff = lambda x: np.array([[3, 0, 1],
                           [0, 4, 2],
                           [1, 2, 3]])

x0 = np.array([1.0, 1.0, 1.0])
print("k\tx[k]\tf(x[k])\terror |∇f|=0")
newton_optimization(ff, dff, ddff, x0);


k	x[k]	f(x[k])	error |∇f|=0
00 [1. 1. 1.]
01 [0.3 0.2 0.1] 2.700000000 0.000000000


##### g) $f(\mathbf x)=- 2 e^{- \left(x_{1} - 1\right)^{2} - \left(x_{2} - 1\right)^{2}}$

In [64]:
fg = lambda x: -2 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2)
dfg = lambda x: np.array([-2 * (2 - 2 * x[0]) * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2), -2 * (2 - 2 * x[1]) * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2)])
ddfg = lambda x: np.array([[-2 * (2 - 2 * x[0])**2 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2) + 4 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2), -2 * (2 - 2 * x[0]) * (2 - 2 * x[1]) * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2)],
                           [-2 * (2 - 2 * x[0]) * (2 - 2 * x[1]) * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2), -2 * (2 - 2 * x[1])**2 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2) + 4 * np.exp(-(x[0] - 1)**2 - (x[1] - 1)**2)]])

x0 = np.array([0.6, 0.75])
print("k\tx[k]\t\t\tf(x[k])\terror |∇f|=0")
newton_optimization(fg, dfg, ddfg, x0)


k	x[k]			f(x[k])	error |∇f|=0
00 [0.6  0.75]
01 [1.32072072 1.20045045] -1.733435048 1.111897076
02 [0.87147898 0.91967436] -1.954583875 0.502410234
03 [1.00618852 1.00386782] -1.999893487 0.024752757
04 [0.99999934 0.99999959] -2.000000000 0.000002637
05 [1. 1.] -2.000000000 0.000000000


array([1., 1.])